### `random embedding`

In [3]:
import tensorflow as tf

In [ ]:
def random_embedding(vocab, embedding_dim):
    embedding_mat = np.random.uniform(-0.25, 0.25, (len(vocab), embedding_dim))
    embedding_mat = np.float32(embedding_mat)
    return embedding_mat

ori_embedding = random_embedding(vocab, embedding_dim)

def embedding(word_ids):
    _word_embedding = tf.Variable(ori_embedding, 
                                  dtype=tf.float32,
                                  trainable = True,
                                  name="_word_embedding")
    word_embedding = tf.nn.embedding_lookup(params=_word_embedding,
                                            ids=word_ids, 
                                            name="word_embedding")
    return word_embedding

In [ ]:
def loss_op():
    if CRF:
        log_likelihood, self.transition_params = crf_log_likelihood(inputs=self.logits, 
                                                                   tag_indices=self.labels,
                                                                   sequence_lengths=self.sequence_lengths)
        self.loss = -tf.reduce_mean(log_likelihood)
        
    else:
        losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, 
                                                                labels=self.labels)
        mask = tf.sequence_mask(self.sequece_length)
        losses = tf.boolean_mask(losses, mask)
        self.loss = tf.reduce_mean(losses)

In [27]:
def mask(inputs,queries=None, keys=None, type=None):
    padding_num = -2**32 + 1
    if type in ("k", "key", "keys"):
        #generate masks
        masks = tf.sign(tf.reduce_sum(tf.abs(keys), axis=-1))  # (N, T_k, d) ==> (N, T_K)
        masks = tf.expand_dims(masks, 1)                       # (N, T_K) ==> (N, 1, T_K)
        masks = tf.tile(masks, [1, tf.shape(queries)[1], 1])   # (N, 1, T_K) ==> (N, T_q, T_K)
        
        paddings = tf.ones_like(inputs) * padding_num
       
        outputs = tf.where(tf.equal(masks, 0), paddings, inputs)
    return (outputs, masks, paddings)
        
        

In [29]:

queries = tf.constant([[[1.],
                        [2.],
                        [0.]]], tf.float32)

keys = tf.constant([[[4.],
                     [0.]]], tf.float32)
inputs = tf.constant([[[4., 0.],
                       [8., 0.],
                       [0., 0.]]], tf.float32)
outputs = mask(inputs, queries, keys, "key")
with tf.Session() as sess:
    _ = sess.run(outputs)
    
    (output, mask, padding) = _
    print output
    print mask
    print padding


[[[ 4.0000000e+00 -4.2949673e+09]
  [ 8.0000000e+00 -4.2949673e+09]
  [ 0.0000000e+00 -4.2949673e+09]]]
[[[1. 0.]
  [1. 0.]
  [1. 0.]]]
[[[-4.2949673e+09 -4.2949673e+09]
  [-4.2949673e+09 -4.2949673e+09]
  [-4.2949673e+09 -4.2949673e+09]]]


### batch_iter

In [ ]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """generate a batch iterator for a dataset"""
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffle_data = data[shuffle_indices]
        else:
            shuffled_data = data
            
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            